# BUILDING A SPAM FILTER WITH NAIVE BAYES
In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. 

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was conveniently put together into the [UCI Machine Learning repository](https://archive.ics.uci.edu/ml/about.html). 

In [1]:
import os
import numpy as np
import pandas as pd

os.listdir()

['.ipynb_checkpoints',
 'data_transform.png',
 'naive_bayesian_classifier.ipynb',
 'SMSSpamCollection']

In [2]:
sms = pd.read_csv('SMSSpamCollection', sep = '\t', header=None)
sms.columns = ['label', 'sms']

sms.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms['label'].value_counts(normalize=True, dropna = False)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

## Methodology approach
We have a good chunk of non-spam messages, here labeled as "ham": we will use this human labeled data as a setup for a simple naive bayes algorithm. This will require us to:  
- Split data into two sets, a training and test set.
- `Training` set will be used as a whole input matrix to build the spam filter.
- `Test` set will be used as a test ground for our algorithm spam/ham estimations, vs the actual results.

As a first shot, we will randomize the dataset, shuffling it in random order.
From there, we will take 80% of the dataset and assign it to the `training` set and the remainder will build the `test` set.

In [4]:
#rearrange the dataframe in random order. The frac parameter is used to address the whole dataframe.
randomized = sms.sample(frac = 1, random_state = 1)
randomized.head()

,label,sms
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [5]:
#prepare a row positional index that will act as a threshold for the two datasets
index = round(0.8 * randomized.shape[0])

#create a training dataframe 
train = randomized.iloc[:index].reset_index(drop = True)
test = randomized.iloc[index:].reset_index(drop = True)

Let's now verify if the percentages of spam / ham are similar to what we have witnessed for the actual beginning dataset.

In [6]:
print("Train dataset composition\n", train.iloc[:,0].value_counts(normalize=True), '\n\n Test dataset composition \n', test.iloc[:,0].value_counts(normalize=True))

Train dataset composition
 ham     0.86541
spam    0.13459
Name: label, dtype: float64 

 Test dataset composition 
 ham     0.868043
spam    0.131957
Name: label, dtype: float64


Looks good! Now we're ready to start building on our naive bayes filter.
Remember that to classify a message we need to create a comparison between the probability of it being spam/ham given the words that build up that specific message.

**P(Spam|W1, W2, ... , Wn) vs P(Ham|W1,W2, ... ,Wn)**

This shall be translated like this view.

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}  

# Crafting the Training Dataset: Vectorization

In order to get to the result above, we will need to transform messages into vectors, so that we are able to:
- Know total number of words in the vocabulary
- make a precise word count and know which words appear how many times, per type of message (spam/ham)

### Data cleaning
We need to clean out the dataframes a bit, so that we can operatePrepare a function that will. Basically, we need to create a single column 

In [7]:
train.head()

,label,sms
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
train

,label,sms
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
4453,ham,"Sorry, I'll call later in meeting any thing re..."
4454,ham,Babe! I fucking love you too !! You know? Fuck...
4455,spam,U've been selected to stay in 1 of 250 top Bri...
4456,ham,Hello my boytoy ... Geeee I miss you already a...


In [9]:
for row in train['sms'].iloc[:5]:
    print(row.split())

['Yep,', 'by', 'the', 'pretty', 'sculpture']
['Yes,', 'princess.', 'Are', 'you', 'going', 'to', 'make', 'me', 'moan?']
['Welp', 'apparently', 'he', 'retired']
['Havent.']
['I', 'forgot', '2', 'ask', 'ü', 'all', 'smth..', "There's", 'a', 'card', 'on', 'da', 'present', 'lei...', 'How?', 'Ü', 'all', 'want', '2', 'write', 'smth', 'or', 'sign', 'on', 'it?']


In [10]:
import re
def cleaner(element):
    #strip it of all punctuation. \W takes all non-words/numbers, that is, symbols.
    element = re.sub(pattern = r'\W', repl = ' ', string = element, flags=re.IGNORECASE)
    
    #put everything into lowercase
    element = element.lower()
    return element


# clean sms column and prepare to generate vocabulary

train['sms'] = train['sms'].apply(cleaner)

vocabulary = []

for row in train['sms']: #iterate the 'sms' series
    message = row.split() #message is a list of words
    for word in message:  #iterate the list and push every single word in it into the vocabulary
        vocabulary.append(word)

#use set to kill duplicates, then go back to a list
vocabulary = list(set(vocabulary))


vocabulary[:5]

['shouted', 'doggin', 'tactful', '89070', 'misss']

## Building final training set
The aim is to build a dataframe that has for each column one word, and for values the count of each word per message/sms. The kind of transformation is represented by this image.

![data_transformation](data_transform.png)  
  
  

In [11]:
#we need to get here
word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }


word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [12]:
#create a separate column with the list version of the words in the sms
train['sms_words'] = train['sms'].str.split()
train.head()

,label,sms,sms_words
0,ham,yep by the pretty sculpture,"[yep, by, the, pretty, sculpture]"
1,ham,yes princess are you going to make me moan,"[yes, princess, are, you, going, to, make, me,..."
2,ham,welp apparently he retired,"[welp, apparently, he, retired]"
3,ham,havent,[havent]
4,ham,i forgot 2 ask ü all smth there s a card on ...,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


As in the example above, we need to create a dictionary where **(key:value) --> (unique_word : word_count)**, for each unique_word in the vocabulary.
- the key in the dictionary will work as a column label. We will therefore end with a dataframe sized as **\[number of sms * vocabulary_length\]**
- The values in the dictionary will need to be lists of zeroes.
- For each set of words in each sms, we will add 1 into the correspondent inde

In [13]:
word_counts_per_sms = {unique_word: np.zeros(len(train['sms_words']), dtype = int) for unique_word in vocabulary}

#for each combination of index and sms, add 1 to the corresponding column value 
for sms_index, sms_words in enumerate(train['sms_words']):
    for word in sms_words:
        word_counts_per_sms[word][sms_index] += 1 #if word index here is a positional argument.
                                                  #It tells which [0,0,0...] has to get a +1.

word_count = pd.DataFrame(word_counts_per_sms)
word_count.head()


,shouted,doggin,tactful,89070,misss,amma,endof,reply,jstfrnd,bites,...,imp,convincing,mca,celebration,lyk,must,ondu,cardiff,burrito,78
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We now need to merge this monster to the train dataset, basically concatenating it on axis = 1 (across rows).

In [14]:
train_cleaned = pd.concat([train, word_count], axis = 1)
train_cleaned.head()

,label,sms,sms_words,shouted,doggin,tactful,89070,misss,amma,endof,...,imp,convincing,mca,celebration,lyk,must,ondu,cardiff,burrito,78
0,ham,yep by the pretty sculpture,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,yes princess are you going to make me moan,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,welp apparently he retired,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,havent,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,i forgot 2 ask ü all smth there s a card on ...,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# **MATHS OF THE NAIVE BAYES SPAM FILTER**

Let's repeat that we need to run a comparison between the two probability values of the equations below, **for each message**.

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations, including alpha as Laplace Smoothing:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}


## Training set
Let's start by calculating the fixed values on the training set.

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>


In [15]:
#using the training set, calculate the fixed values of the formula
spam_messages = train_cleaned[train_cleaned['label'] == 'spam']
ham_messages = train_cleaned[train_cleaned['label'] == 'ham']

#probability of a message being spam/ham
train_p_spam = train_cleaned['label'].value_counts(normalize = True)['spam']
train_p_ham = 1 - train_p_spam

#total number of words in spam/ham messages. Select a dataframe slice and sum results for all series. sum 
n_spam = sum(spam_messages.iloc[:,3:].sum())
n_ham = sum(ham_messages.iloc[:,3:].sum())

#total number of words in the vocabulary
n_vocabulary = len(vocabulary)

#Laplace smoothing
alpha = 1

In [16]:
train_cleaned.head(1)

,label,sms,sms_words,shouted,doggin,tactful,89070,misss,amma,endof,...,imp,convincing,mca,celebration,lyk,must,ondu,cardiff,burrito,78
0,ham,yep by the pretty sculpture,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now onto
P(w<sub>i | Spam</sub>) / P(w<sub>i | Ham</sub>).  
Given we already have the fixed values, looking at the formula.

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


This is about calculating **N<sub>W<sub>i</sub> | Spam</sub>** , that is:
- N(<sub>cricket | Spam</sub>) --> The number of times the word "cricket" appears in Spam messages
- N(<sub>desparate | Spam</sub>) --> The number of times the word "desperate" appears in Spam messages
- ...

#instantiate two dictionaries that will store the word count, per each word, given spam/ham
parameters_spam = {unique_word: 0 for unique_word in vocabulary}
parameters_ham = {unique_word: 0 for unique_word in vocabulary}

In [17]:
#prepare two series, that collect total word counts per spam / ham messages
spam_word_count = spam_messages.iloc[:,3:].sum()
ham_word_count = ham_messages.iloc[:,3:].sum()

#create a probabilities dataframe putting together the two series
probabilities = pd.concat([spam_word_count, ham_word_count], axis = 1)
probabilities.columns = ['spam','ham']

#calculate probability values for each word, given they belong to a spam/ham messageb
probabilities['p_w_given_spam'] = (probabilities['spam'] + alpha) / (n_spam + alpha * n_vocabulary)
probabilities['p_w_given_ham'] = (probabilities['ham'] + alpha) / (n_ham + alpha * n_vocabulary)

probabilities.head()

,spam,ham,p_w_given_spam,p_w_given_ham
shouted,0,2,0.000044,0.000046
doggin,1,0,0.000087,0.000015
tactful,0,1,0.000044,0.000031
89070,2,0,0.000131,0.000015
misss,0,1,0.000044,0.000031


In [18]:
probabilities.shape

(7783, 4)

## Classifying A New Message  
Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., wn).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) \> P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) \< P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.  
    
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}
    
**Given we have already calculated P|spam, for each new message we will just need to consider the words and calculate the product of them.**


In [19]:
def classify(message):
    #message is a string#
    words = cleaner(message).split() #clean message and convert into a list of words
    
    #take fixed values of the training set
    p_spam = train_p_spam
    p_ham = train_p_ham
    
    #calculate number of times each word appears in Spam messages
    for word in words:
        if word in probabilities.index:
            p_spam *= probabilities.loc[word]['p_w_given_spam'] #multiply p_spam by the p_w_given spam taken from the df row
            p_ham *= probabilities.loc[word]['p_w_given_ham']

    print('P(Spam|message):', p_spam)
    print('P(Ham|message):', p_ham)      
            
    if p_spam > p_ham:
        print('This message is spam')
    elif p_spam < p_ham:
        print('This message is NOT spam')
    else:
        print('Equal probabilities, needs human classification')

In [20]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')
print('\n')
classify("Sounds good, Tom, then see u there")


P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
This message is spam


P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
This message is NOT spam


# Measuring the Spam Filter's Accuracy
The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [21]:
def classify_test_set(message):
    #message is a string#
    words = cleaner(message).split() #clean message and convert into a list of words
    
    #take fixed values of the training set
    p_spam = train_p_spam
    p_ham = train_p_ham
    
    #calculate number of times each word appears in Spam messages
    for word in words:
        if word in probabilities.index:
            p_spam *= probabilities.loc[word]['p_w_given_spam'] #multiply p_spam by the p_w_given spam taken from the df row
            p_ham *= probabilities.loc[word]['p_w_given_ham']
            
    if p_spam > p_ham:
        return 'spam'
    elif p_spam < p_ham:
        return 'ham'
    else:
        return 'needs human classification'


test.head()

,label,sms
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


In [22]:
#apply classify function to whole dataframe and store results in new columns
test['classification'] = test['sms'].apply(classify_test_set)


#measure accuracy as number of messages correctly guessed vs total classified messages
guessed_right = 0
for row in test.itertuples():
    if row[1] == row[3]: #column positional indexes to access the values of cols 'label' and 'classification'
        guessed_right += 1

#finalize accuracy calculation
accuracy = guessed_right / len(test)
print('The number of correctly guessed messages by the spam filter is {:.2%}'.format(accuracy))

The number of correctly guessed messages by the spam filter is 98.74%


Excellent result! let's take a quick look at the messages guessed wrong.

In [23]:
pd.set_option('max_colwidth', 200)

test[test['classification'] != test['label']]

,label,sms,classification
114,spam,Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net,ham
135,spam,More people are dogging in your area now. Call 09090204448 and join like minded guys. Why not arrange 1 yourself. There's 1 this evening. A£1.50 minAPN LS278BB,ham
152,ham,Unlimited texts. Limited minutes.,spam
159,ham,26th OF JULY,spam
284,ham,Nokia phone is lovly..,spam
293,ham,"A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d gi...",needs human classification
302,ham,No calls..messages..missed calls,spam
319,ham,"We have sent JD for Customer Service cum Accounts Executive to ur mail id, For details contact us",spam
504,spam,"Oh my god! I've found your number again! I'm so glad, text me back xafter this msgs cst std ntwk chg £1.50",ham
546,spam,"Hi babe its Chloe, how r u? I was smashed on saturday night, it was great! How was your weekend? U been missing me? SP visionsms.com Text stop to stop 150p/text",ham


It looks like the large majority of these messages are wrongly guessed because they contained of a good number of plausible words used in 'ham' messages.  
In simple terms, if 90% of the words used in the message are used and available in non-spam messages, the bayesian filter will classify them as being good messages - while ultimately they are holding in the tail of the message the real spam content. 

This technique to trick bayesian filters is also known as **[Bayesian Poisoning](https://en.wikipedia.org/wiki/Bayesian_poisoning).**